# Web Crawling Part.2

## 네이버 뉴스 기사 크롤링

In [1]:
from bs4 import BeautifulSoup 
import requests #HTTP 요청을 보내는 모듈 
import re #정규 표현식 사용하기 위한 모듈
import sys # 이 sys 모듈을 이용해서 파이썬 라이브러리가 설치되어 있는 디렉터리를 확인할 수 있다.
import pprint #pprint 모듈은 인터프리터가 읽을 수 있는 방식으로 내장 객체나 사용자 정의
#객체를 인쇄하는 것을 보다 정교하게 제어할 수 있게 합니다. 결과가 한 줄보다 길면 
#"예쁜 프린터"가 줄 바꿈과 들여쓰기를 추가하여 데이터 구조를 보다 명확하게 나타냅니다
import pandas as pd

In [2]:
sys.path

['C:\\Users\\SAMSUNG\\Desktop\\크롤링 자료\\4주차_크롤링실습',
 'C:\\Users\\SAMSUNG\\Anaconda3\\python37.zip',
 'C:\\Users\\SAMSUNG\\Anaconda3\\DLLs',
 'C:\\Users\\SAMSUNG\\Anaconda3\\lib',
 'C:\\Users\\SAMSUNG\\Anaconda3',
 '',
 'C:\\Users\\SAMSUNG\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\SAMSUNG\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\SAMSUNG\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\SAMSUNG\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\SAMSUNG\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\SAMSUNG\\.ipython']

http://pythonkim.tistory.com/91 #pprint

In [3]:
#전체적인 구조 설명 - 먼저 get_news함수를 정의하고 이를 통해 news_detail을 불러온다. 그리고 불러온 news_detail에서 
#필요한 정보를 파싱해서
#데이터 프레임으로 만들어주는 것이다. news_detail 함수를 만들고 while문을 돌리기 위한 확인 작업 - while문 돌리기

def get_news(n_url): 

    news_detail = [] 

    print(n_url) 

    breq = requests.get(n_url) #request 모듈의 get 합수를 통해 http에 연결 요청을 해서 가져온 url을 breq 변수에 저장

    bsoup = BeautifulSoup(breq.content, 'html.parser') #content함수를 통해 breq에 저장된 url의 html소스를 가져오고, 그것을 html.parser를 통해 파싱한다.
    #bsoup에는 url에 저장된 소스의 내용을 파싱한 상태가 저장되는 것이다.
 

    # html 파싱 

    title =bsoup.findAll("h3",{"id":"articleTitle"})[0].get_text() #bsoup에 있는 소스 중 h3태그의 id속성의 속성값이 articleTitle인 것의
    #첫 번째 텍스트를 가져오면 title이 된다. 이것은 실제로 소스를 확인하면 알 수 있다. title이 있는 부분을 찾고 해당하는 태그, 속성, 속성 값을 
    #확인해보면 알 수 있다. 어찌됐든 이 소스에서 제목에 해당하는 부분은 저 위치에 있다.

    news_detail.append(title) #파싱을 통해 가져온 제목을 append함수를 통해 빈 리스트인 news_detail에 추가한다.

 

    # 날짜 파싱

    pdate = bsoup.findAll("span",{"class":"t11"})[0].get_text() #위에 같은 방법으로 수행한다.

    news_detail.append(pdate) #날짜 역시 news_detail에 추가한다.

 

    # 기사 본문 크롤링 

    _text = (bsoup.findAll("div",{"id":"articleBodyContents"})[0].get_text()).replace('\n', " ") #위 과정과 동일.
    #replace 함수는 괄호 안 첫번째에 있는 것을 뒤에 있는 내용으로 바꾼다는 것이다. 예를 들어 a.replace(","," " )    
    # a = 123,234,35일 경우 위의 replace함수를 적용하면
    # 123 234 35로 바뀐다.

    btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "") 
    #괄호안의 내용이 나오면 ""로 바꾸라는 것이니 그냥 저 내용을 없애라는 의미로 해석하면 된다.

    news_detail.append(btext.strip()) #btext 즉, 텍스트에서 아래 오류 내용이 제거된 상태의 텍스트를 news_detail에 추가한다.

 

    # 신문사 크롤링
    pcompany =bsoup.findAll("address",{"class":"address_cp nclicks(fot.presscr)"})[0].get_text()
    #pcompany = bsoup.select('#footer address')[0].a.get_text() 

    news_detail.append(pcompany) 

 

    return news_detail #결국 이 예약어를 통해 가져오려는 값은 news_detail이다. 
    #우리가 url을 이 함수에 넣어주면 그 기사 제목, 날짜, 신문사, 내용이 news_detail에 리스트 형태로 넣어져서 나온다.

In [4]:
columns = ['날짜','신문사', '제목','내용'] #하나씩 출력되는 개별 기사마다의 news_detail을 통합시키기 위해 데이터프레임을 만든다. 컬럼 설정 
df = pd.DataFrame(columns=columns) #pandas 수업 때 했던 것으로 컬럼 파라미터에 위에 설정한 컬럼을 넣어준다는 의미다.

In [5]:
# 쿼리에 검색어를 입력하고 검색 시작날짜부터 끝 날짜까지를 입력
#여러분들이 실제로 이 네이버 웹 크롤러를 사용할 때 사실상 여기만 건드리면 된다. 쿼리는 여러분들이 검색할 검색어

query = "청소기"   # url 인코딩 에러는 encoding parse.quote(query) 

s_date = "2018.11.05" # 네이버에서 뉴스 검색옵션에서 기간 설정할 때 시작 날짜

e_date = "2018.11.11" #끝 날짜

s_from = s_date.replace(".","") #실제 네이버 뉴스 검색한 페이지의 url을 보면 시작 날짜가 나오고 뒤에 부분에 또 시작날짜에서'.'이 사라진 형태가 나온다. 그래서 '.'을 없애준다.
# 아래 url을 만드는 부분 참고하세요!

e_to = e_date.replace(".","") #위와 마찬가지


page = 1 #페이지가 실제 페이지를 의미하는 것이 아니고 기사 순서대로 번호를 의미한다. 우리가 찾는 첫번째 기사라고 보면 된다.

#1이라고 적혀있는 것은 #1끼리 실행하고 #2끼리 실행하세요!
#1 실행하고 #2도 실행하면 안돼요!!! #1실행했으면 #2는 넘어가세요!ㅋㅋㅋ사실 이 뒤에부터 while문 뒤까지는 다 생략해도 되는 부분이에요. 위에 함수 정의하고 바로 while문 실행하면 크롤링됩니다! 그냥 #1이랑 #2차이 보기위해 해둔 거고 while문 두 개 중에 하나만 실행하면 돼요!
그리고 중간과정은 생략해도 됩니다. 중간과정은 그냥 코드 확인을 위해 넣은 거에요!
#1과 #2는 url구성을 조금씩 다르게 해서 결과값이 달라집니다.

In [6]:
#1

url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=1&field=1&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page) 
#이 부분 잘 이해해야 한다. 파이썬은 편리하게 '+'를 통해 연결해줄 수 있다. 인용부호는 네이버 뉴스 검색 페이지에서 1,2,3,4....모든 페이지 url의 공통된 부분에 대해서 사용한다.
#각 페이지마다 차이점에 대해서만 우리가 바꿔주는 것이다. 따라서 검색어와 기간 설정 내용 등을 넣어주는 것이다. 
#이렇게 직접 만든 url을 requests 모듈의 get함수를 통해 http에 요청하여 파이썬에 가져오는 작업을 거치고 가져온 url의 소스를 파싱하는 것이다.
# 여러분들이 직접 건드려야 하는 부분이기도 하다. 실제로 데이터를 긁어오고 싶은 사이트의 url을 보고 페이지를 넘겨가면서 url의 공통적인 부분과
# 페이지마다 다른 부분을 구분해서 위에 만든 것처럼 url을 만들어주면 된다.

    
header = { 

        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36' 

    } 
#환경을 고정시켜준것이다. windows 환경으로 세팅한 것이다. 어디서 켜졌는지 세팅을 해주면 크롤러가 안정적이게 되는 것이다. 헤더를 바꿔주면 나라도 바꿔줄 수 있다.
# 사실 없어도 되는 부분이긴 하다.

req = requests.get(url,headers=header) 
#바로 위에 설명했듯이 requests 모듈의 get함수를 통해 우리가 직접 만들어준 url을 jupyter에 연결해준다. 그리고 req에 저장한다.


### 아래 while문을 위한 코드별 확인 과정

In [ ]:
req

In [ ]:
cont = req.content #가져온 req에서 content함수를 통해 소스 내용을 가져와 cont에 저장

In [ ]:
cont

In [ ]:
soup = BeautifulSoup(cont, 'html.parser') #beautifulsoup을 통해 파싱 작업! cont에 저장된 소스를 파싱한 것을 soup에 저장. 
# cont와 soup은 완전 다르다. 구분해야 한다. 파싱을 진행한 상태라는 것을 염두해야 한다. 파싱을 안 한 상태에서 아래와 같이 태그, 속성 등을 가져오는게
# 안 된다.

In [ ]:
soup

In [ ]:
temp=soup.findAll("a",{"class":"_sp_each_url"})[0] #a 태그의 class 속성의 속성 값이 _sp_each_url 인 text를 구하고 [0] 즉, 그 text의 slicing 된 것들 중 첫번째 값을 모두 가져온다.

In [ ]:
temp #실행해보면 이 부분에 이 url의 소스에 해당 url이 그대로 포함되어 있다는 것을 알 수 있다. 우리가 url을 그대로 안 쓰는 이유는 위에 사용해 온
#beautifulsoup이라는 라이브러리가 소스에 접근해야 사용가능한 것이기 때문이다. url을 알고 있어도 소스에 나와있는 url에 접근해야 한다. 

In [ ]:
temp.attrs["href"].startswith("https://news.naver.com") #startswith함수를 통해 temp의 href 속성 값이 https://news.naver.com로 시작하는지 확인
#true가 나오면 https://news.naver.com로 시작한다는 것이다. temp는 url을 포함한 다른 내용들이 있기 때문에 url만 가져오기 위해 
#'href'태그에 접근한다. a 태그의 class 속성 값이 _sp_each_url인 것을 가져온 것과 속성이 'href'인 것을 가져오는 것을 구분해야 한다. 
# 위에 temp를 보면 알겠지만 전자가 더 많은 내용을 포함한다. 속성이 href인 것은 ""안에 있는 url만 포함하고 있다는 것을 알 수 있다.
#1번으로 실행시키면 false가 나오는데 당황하지 마세요! 첫번째 기사의 url이 naver로 시작하지 않아서 그렇다. 위에 temp에서 확인할 수 있다.
#결국 처음에 url을 어떻게 구성해주느냐에 따라 가져오는 결과값이 달라진다.

In [ ]:
temp.attrs["href"] #따라서 속성 값과 속성에 모두 접근해서 사용할 수 있는 attrs 함수를 통해 'href' 속성에 해당하는 text를 가져온 것이다.
#드디어 소스에 있는 url을 가져왔다.

### 위 코드들로 각 코드가 무엇을 의미하는지 알았으면 while문을 실행해볼게요!

In [7]:

#위 url이 #1인 것과 대응
#1
#위에 확인 작업한 것들을 생각하며 아래 코드 진행을 잘 보세요! while문에 넣을 내용을 위에 개별 코드로 설명한 것 뿐입니다.

while True:

    
    print(page) #위에서 page = 1로 넣어뒀죠?! 기억하세요
    
    url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=1&field=1&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page) 
    #위와 동일하게 만든 url을 넣어줍니다.

    header = { 

        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36' 

    } 
    req = requests.get(url,headers=header) 

    print(url) 

    cont = req.content 

    soup = BeautifulSoup(cont, 'html.parser') 
    #위에 한 거랑 똑같아요! 이해 안되면 위에 읽어보세요


    if soup.findAll("a",{"class":"_sp_each_url"}) == [] : #위에 개별 코드 설명 참고

        break 

    for urls in soup.findAll("a",{"class":"_sp_each_url"}): #위에 코드 설명 참고
        #반복문 실행 - url의 소스 중 위에 temp에 해당하는 부분을 가져오는 반복문이다. 위에 temp는 잊으세요. 어차피 위에는 확인을 위한 것이고
        #우리는 실제 이 while문을 돌리기 위해 예시로 설명한 것 뿐입니다. 암튼 위에 temp에 해당하는 소스를 가져옵니다. 여기서는 urls로 명명했어요.

        try : #urls를 가져온 상태로 try문을 돌려요
           

            if urls.attrs["href"].startswith("https://news.naver.com"): #위에 temp.attrs['href']랑 같은 내용이에요

                print(urls.attrs["href"]) #아무튼 위에 true가 뜨면 뽑아져 나온 url을 프린트하라는 거에요

                news_detail = get_news(urls.attrs["href"]) 
                #위에 나오는 urls.attrs['herf']를 통해 나오는 뉴스 기사 url을 처음에 정의했던 get_news() 함수에 넣어주는 것이다.
                #get_news()함수에는 위에 써져 있듯이 url을 넣는다. 

                df=df.append(pd.DataFrame([[news_detail[1], news_detail[3], news_detail[0], news_detail[2]]],columns=columns)) #news_detail에 만들어진 리스트를 데이터 프레임으로 만들어준다.
                #대괄호 속의 숫자는 데이터 프레임에 넣는 순서를 뜻한다. 즉, 날짜, 신문사, 제목, 본문 순서대로 데이터프레임에 넣는다.

        except Exception as e: #에러가 날 경우를 대비한 것

            print(e)  # e 라는 에러가 나면 그 에러 내용 e를 프린트해라

            continue #그리고 계속 진행해라. 다시 말해 에러가 뜨면 에러 내용을 출력하고 그냥 무시하고 계속해라

    page += 10   #한 페이지에 기사가 10개씩 있기 때문에 다음 페이지로 넘어갈 때 10을 더해주는 것이다.



1
https://search.naver.com/search.naver?where=news&query=청소기&sort=1&field=1&ds=2018.11.05&de=2018.11.11&nso=so%3Ar%2Cp%3Afrom20181105to20181111%2Ca%3A&start=1
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=015&aid=0004042701
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=015&aid=0004042701
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=088&aid=0000561706
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=088&aid=0000561706
11
https://search.naver.com/search.naver?where=news&query=청소기&sort=1&field=1&ds=2018.11.05&de=2018.11.11&nso=so%3Ar%2Cp%3Afrom20181105to20181111%2Ca%3A&start=11
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=008&aid=0004130035
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=008&aid=0004130035
21
https://search.naver.com/search.naver?where=news&query=청소기&sort=1&field=1&ds=2018.11.05&de=2018.11.11&nso=so%3Ar%2Cp%3Afrom20181105to20181111%2Ca%3A&start=21
htt

In [ ]:
#2

while True:

    
    print(page) 
    
    url = "https://search.naver.com/search.naver?&where=news&query=" + query + "&sort=1&photo=0&field=0&reporter_article=&pd=3&ds=" + s_date + "&de=" + e_date + "&docid=&nso=so:dd,p:from" + s_from + "to" + e_to + ",a:all&mynews=0&start=" + str(page) + "&refresh_start=0"

    header = { 

        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36' 

    } 
    req = requests.get(url,headers=header) 

    print(url) 

    cont = req.content 

    soup = BeautifulSoup(cont, 'html.parser') 


    if soup.findAll("a",{"class":"_sp_each_url"}) == [] :

        break 

    for urls in soup.findAll("a",{"class":"_sp_each_url"}): 

        try : 
           

            if urls.attrs["href"].startswith("https://news.naver.com"): 

                print(urls.attrs["href"]) 

                news_detail = get_news(urls.attrs["href"]) 

                    

                df=df.append(pd.DataFrame([[news_detail[1], news_detail[3], news_detail[0], news_detail[2]]],columns=columns))

        except Exception as e: 

            print(e)  

            continue 

    page += 10   

# https://docs.python.org/ko/3/tutorial/errors.html - except as e: 에러와 예외 참고

In [8]:
news_detail #위의 while문에 의해 get_news함수에 url을 넣어서 나온 결과값이죠! page가 처음에 1로 설정되어 있으니 실행된 첫번째 기사의 
#모든 내용이 출력될 거에요

['위메프 에어팟, 오후 11시에도 완판… 다음 판매 제품은 다이슨 청소기?',
 '2018.11.05. 오후 11:43',
 '사진=위메프 공식 인스타그램위메프가 애플 에어팟을 특가 판매했다.5일 위메프는 이날 오전 11시와 오후 11시에 애플 에어팟 1천개를 각 500개 씩 한정수량으로 11만 111원에 판매했다. 에어팟은 1~2분 만에 제품이 품절되는 등 큰 인기를 끌었다.위메프는 에어팟에 앞서 맥 립스틱, 하기스 기저귀 3팩, 페레로로쉐 T8+T8 등을 판매했다.위메프의 행사는 11일까지 진행된다. 내일(6일)은 휘닉스파크 리프트권(1,111원/2000개), 7일에는 BHC 치킨 뿌링클+콜라(1,111원/5000개)를 저렴한 가격에 선보인다.8일에는 도미노 포테이토 M(1,111원/5000개), 9일에는 다이슨 V8 앱솔루트(111,111원/100개) 등이 판매될 예정이다.디지털콘텐츠팀 multi@',
 'Copyright ⓒ 부산일보 All Rights Reserved.']

In [9]:
df #와 드디어 다 왔다...그래서 while문에 보면 news_detail에 있는 내용을 데이터 프레임에 저장했고 그것을 반복문을 통해 모든 기사의 내용들이
#추가되었습니다. 축하해요ㅎㅎ 여기까지 오느라ㅋㅋㅋㅋ

,날짜,신문사,제목,내용
0,2018.11.09. 오전 11:59,Copyright ⓒ 한경닷컴 All Rights Reserved.,다이슨V8 앱솔루트 청소기가 11만원? 위메프 이어 11번가까지,[ 김소연 기자 ] /사진=위메프다이슨V8 앱솔로트 청소기를 100대 한정으로 1...
0,2018.11.09. 오전 10:23,Copyright ⓒ 매일신문 All Rights Reserved.,"위메프, 다이슨 V8 앱솔루트 청소기 100대 ""11만1천111원 판매""…9일 오전...","일월온수매트 퀸사이즈(1000개, 9만9천111원), 나이키 티셔츠(20000개, ..."
0,2018.11.08. 오후 6:00,Copyright ⓒ MoneyToday All Rights Reserved.,"[단독]""7시에 거실만 청소해줘!""…말귀 알아듣는 청소기","[머니투데이 이정혁 기자] [LG전자 음성인식 로봇청소기 출시, 원하는 장소만 음성..."
0,2018.11.07. 오전 11:39,Copyright ⓒ 한경닷컴 All Rights Reserved.,"위메프, '블프' 대규모 할인 행사…다이슨 청소기 11만1111원",/ 사진=위메프이커머스 기업인 위메프가 애플 에어팟에 이어 파격적인 할인 행사를 연...
0,2018.11.05. 오후 11:43,Copyright ⓒ 부산일보 All Rights Reserved.,"위메프 에어팟, 오후 11시에도 완판… 다음 판매 제품은 다이슨 청소기?",사진=위메프 공식 인스타그램위메프가 애플 에어팟을 특가 판매했다.5일 위메프는 이날...
